In [28]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
import time

# Homework 4

## 1) Does basic house information reflect house's description?

In this assignment we will perform a clustering analysis of house announcements in Rome from Immobiliare.it. Be careful you may notice that the announcement is written in Italian. Don't worry about it, you don't need to understand what's in it.

![alt text](https://images.adsttc.com/media/images/5b76/1d5f/f197/cc80/ea00/01b2/slideshow/T16_1158b.jpg?1534467412)

### Scraping
The first thing that we have to do is to create our dataset. The website that we will scrape is: [immobiliare.it](https://www.immobiliare.it). In particular, we retrieve announcements starting from this [link](https://www.immobiliare.it/vendita-case/roma/?criterio=rilevanza&pag=1).


#### 1) Information
The first matrix will have this format: <img src="https://latex.codecogs.com/gif.latex?$m_{ij}&space;=&space;value$" title="$m_{ij} = value$" /> where <img src="https://latex.codecogs.com/gif.latex?$i&space;\in&space;\{announcement_1,&space;...,&space;announcement_n\}$" title="$i \in \{announcement_1, ..., announcement_n\}$" /> and <img src="https://latex.codecogs.com/gif.latex?$j&space;\in&space;\{price,&space;locali,&space;superficie,&space;bagni,&space;piano&space;\}$" title="$j \in \{price, locali, superficie, bagni, piano \}$" />. *n* is the number of the announcements. It's possible that not all the announcements will have all the fields mentioned above, if it's the case don't take it into account. 

We create an empty dataframe where we will store all the informations.

In [2]:
df= pd.DataFrame(index=['price', 'locali', 'superficie', 'bagni', 'piano'] )
df

""
price
locali
superficie
bagni
piano


We take informations of all the house in the first $700$ pages (every page contains $25$ announcements).

In [4]:
count=1
for i in range (1,700):
    # requests.get helps us to enter the page. 
    # Then BeatifulSoup gives us the xml format.
    content= requests.get("https://www.immobiliare.it/vendita-case/roma/?criterio=rilevanza&pag="+str(i))
    soup = BeautifulSoup(content.text, "lxml")
    for j in soup.findAll("div", class_="listing-item_body"):
        annuncio='annuncio_'+str(count)
        locali= None
        superficie= None
        bagni = None
        piano = None
        try:
            price=j.find("li", class_="lif__item lif__pricing").contents[-1]
            price= int(price.replace('\n', ' ').replace('€', ' ').replace('.', '').replace(' ', ''))
        except:
            price=None
        resto=j.findAll("li", class_="lif__item")
        for k in range (len(resto)):
            if (len((re.findall(string=str(resto[k]), pattern=r'locali'))))==1:
                locali=resto[k].span.contents[0].replace('\xa0', ' ')
            if (len((re.findall(string=str(resto[k]), pattern=r'superficie'))))==1:
                superficie=int(resto[k].span.contents[0].replace('\xa0', ' ').replace('.', ''))
            if (len((re.findall(string=str(resto[k]), pattern=r'bagni'))))==1:
                bagni=resto[k].span.contents[0].replace('\xa0', ' ')
            if (len((re.findall(string=str(resto[k]), pattern=r'piano'))))==1:
                piano=resto[k].abbr.contents[0].replace('\xa0', ' ').replace('\n', ' ') 
    
        df[annuncio]=[price, locali, superficie, bagni, piano]
        count+=1

In [5]:
df

,annuncio_1,annuncio_2,annuncio_3,annuncio_4,annuncio_5,annuncio_6,annuncio_7,annuncio_8,annuncio_9,annuncio_10,...,annuncio_17464,annuncio_17465,annuncio_17466,annuncio_17467,annuncio_17468,annuncio_17469,annuncio_17470,annuncio_17471,annuncio_17472,annuncio_17473
price,192000,225000,359000,800000,3400000,950000,250000,1090000,2150000,2500000,...,155625.0,330000,189000,469000,154740.0,375000,865000,160000,128400,70000
locali,1-5,2,4,5+,5+,5,4,5+,4,5+,...,NaN,3,3,4,NaN,4,5+,2,3,3
superficie,46,50,130,220,500,170,110,190,150,300,...,107.0,94,87,110,250.0,160,350,50,116,60
bagni,None,1,3,3,3+,1,2,2,1,3,...,NaN,2,1,2,NaN,2,3+,1,2,1
piano,None,1,3,4,None,3,A,2,4,2,...,NaN,1,R,2,NaN,1,None,T,1,S


In [6]:
df2=df.dropna(axis='columns')

In [7]:
df2

,annuncio_2,annuncio_3,annuncio_4,annuncio_6,annuncio_7,annuncio_8,annuncio_9,annuncio_10,annuncio_11,annuncio_12,...,annuncio_17460,annuncio_17461,annuncio_17462,annuncio_17465,annuncio_17466,annuncio_17467,annuncio_17469,annuncio_17471,annuncio_17472,annuncio_17473
price,225000,359000,800000,950000,250000,1090000,2150000,2500000,650000,680000,...,109000,249000,108750,330000,189000,469000,375000,160000,128400,70000
locali,2,4,5+,5,4,5+,4,5+,5+,3,...,2,1,2,3,3,4,4,2,3,3
superficie,50,130,220,170,110,190,150,300,193,90,...,60,47,72,94,87,110,160,50,116,60
bagni,1,3,3,1,2,2,1,3,3,1,...,1,1,1,2,1,2,2,1,2,1
piano,1,3,4,3,A,2,4,2,4,3,...,2,11+,T,1,R,2,1,T,1,S


In [32]:
count=1
for i in range (1,700):        
    time.sleep(4)
    # requests.get helps us to enter the page. 
    # Then BeatifulSoup gives us the xml format.
    content= requests.get("https://www.immobiliare.it/vendita-case/roma/?criterio=rilevanza&pag="+str(i))
    soup = BeautifulSoup(content.text, "lxml")
    urls=[]
    for j in soup.findAll("div", class_="listing-item_body"):
        for link in j.find_all('a'):
            if link.get('href') is not None and (link.get('href').startswith('https')):
                url=(link.get('href'))
                content=requests.get(url)
                soup = BeautifulSoup(content.text, "lxml")
                desc=soup.find("div", class_="col-xs-12 description-text text-compressed")
                op = open(r'C:\Users\Egon\Desktop\Universita\ADM\HW4\doc\annuncio_' + str(count) + '.tsv', 'w', encoding="utf-8")
                op.write(str(desc.contents[1]))
                op.close()
        count+=1

AttributeError: 'NoneType' object has no attribute 'contents'

In [22]:
content= requests.get("https://www.immobiliare.it/vendita-case/roma/?criterio=rilevanza&pag=1")
soup = BeautifulSoup(content.text, "lxml")
count=1
doc=1
urls=[]
for j in soup.findAll("div", class_="listing-item_body"):
    for link in j.find_all('a'):
        if link.get('href') is not None and (link.get('href').startswith('https')):
            urls.append(link.get('href'))
            print(link.get('href'), doc)
    doc+=1
print(urls)

https://www.immobiliare.it/53131931-Vendita-Bilocale-viale-Italo-Calvino-Roma.html 2
https://www.immobiliare.it/69866648-Vendita-Quadrilocale-via-Mario-Ridolfi-32-Roma.html 3
https://www.immobiliare.it/68373749-Vendita-Appartamento-via-dei-Gandolfi-Roma.html 4
https://www.immobiliare.it/68083741-Vendita-Villa-via-Casalmonferrato-Roma.html 5
https://www.immobiliare.it/65062100-Vendita-Appartamento-piazza-Cavour-Roma.html 6
https://www.immobiliare.it/63857680-Vendita-Quadrilocale-viale-della-Grande-Muraglia-Roma.html 7
https://www.immobiliare.it/67959361-Vendita-Appartamento-piazza-Del-Fante-Roma.html 8
https://www.immobiliare.it/67012733-Vendita-Attico-Mansarda-via-della-Frezza-Roma.html 9
https://www.immobiliare.it/61733354-Vendita-Appartamento-via-Dandolo-Roma.html 10
https://www.immobiliare.it/70043828-Vendita-Appartamento-via-del-Calice-Roma.html 11
https://www.immobiliare.it/65779135-Vendita-Trilocale-via-dei-Banchi-Vecchi-Roma.html 12
https://www.immobiliare.it/64401288-Vendita-Vi

In [437]:
count=0
for url in urls:
    content=requests.get(url)
    soup = BeautifulSoup(content.text, "lxml")
    desc=soup.find("div", class_="col-xs-12 description-text text-compressed")
    op = open(r'C:\Users\Egon\Desktop\Universita\ADM\HW4\doc\doc_' + str(count) + '.tsv', 'w', encoding="utf-8")
    op.write(str(desc.contents[1]))
    op.close()
    count+=1

In [429]:
desc.contents[1]

<div>
                                PAPILLO EUR in elegante complesso residenziale rifinitissimo bilocale composto da soggiorno con angolo cottura, stanza da letto bagno e ampio balcone . con Rifiniture di pregio, pavimenti in parquet / grees, infissi in legno con vetro camera e porte in noce, grate nel salone, riscaldamento termoautonomo con caldaia centralizzata, aria condizionata, videocitofono, porta blindata, serramenti elettrici con chiusura centralizzata, antenna satellitare, isolamento termo acustico, pannelli solari e fotovoltaici , rilevatori elettronici di gas. Tutte le camere sono fornite di impianto antifurto, presa antenna satellitare e presa telefonica.<br/>
<br/>
ORARI lunedi chiusi<br/>
 martedi 10:00-17:00<br/>
 mercoledi 10:00-17:00<br/>
<hr/>giovedi 10:00-17:00<br/>
<hr/>venrdi 10:00-17:00<br/>
<hr/>sabato 10:00-17:00<br/>
 domenica 10:00-13:00
            </div>

In [ ]:
for i in range(25):
    op = open(r'C:\Users\Egon\Desktop\Universitaa\ADM\HW4\doc\doc_' + str(i) + '.tsv', 'w', encoding="utf-8")
    op.write(desc.contents[1])
    op.close()